<a href="https://colab.research.google.com/github/Melvinchen0404/Chess_engine_v2/blob/main/sapientia_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chess python-chess ipython datasets
!apt-get install stockfish

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
stockfish is already the newest version (14.1-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
import os
import sys
import argparse
import chess
import chess.engine
import chess.svg
from IPython.display import display, SVG, clear_output
import ipywidgets as widgets
from openings import process_moves, uci_to_san, check_openings, get_user_moves
from gm_database import fetch_gm_data, process_gm_data
from engines import fetch_komodo_data, fetch_stockfish_data
from endgame import fetch_endgame_tablebase_data, best_endgame_move

# ANSI escape code for bold text
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset formatting to normal

# Global variables
current_uci_sequence = []
current_san_sequence = []
# Sequence mode flag and moves
sequence_mode = False
sequence_moves = []
board = chess.Board()  # Initialize the board
chessboard_output = widgets.Output()  # Create a separate output area for rendering the chessboard (this will be updated separately)

# Create widgets for command input and submit button
input_box = widgets.Text(placeholder='Enter your move or command', description='Command:')
submit_button = widgets.Button(description="Submit")
reset_button = widgets.Button(description="Reset")
undo_button = widgets.Button(description="Undo")
output_area = widgets.Output()

# Layout the buttons horizontally
button_layout = widgets.HBox([submit_button, reset_button, undo_button])

button_style = {'description_width': 'initial'}
submit_button.layout.width = '7%'  # Make the button 5% of the parent container width
reset_button.layout.width = '7%'  # Same for the reset button
undo_button.layout.width = '7%'  # Same for the undo button

submit_button.layout.height = '20px'  # Adjust height to make them shorter
reset_button.layout.height = '20px'  # Same height for reset button
undo_button.layout.height = '20px'  # Same height for undo button

submit_button.style.font = 'Arial 5pt'  # Adjust font size to smaller (8pt)
reset_button.style.font = 'Arial 5pt'  # Adjust font size to smaller (8pt)
undo_button.style.font = 'Arial 5pt'  # Adjust font size to smaller (8pt)

# Optional: Adjust font weight if needed
submit_button.style.font_weight = 'normal'
reset_button.style.font_weight = 'normal'
undo_button.style.font_weight = 'normal'

# Combine input box, buttons, and output area into a vertical layout
layout_container = widgets.VBox([input_box, button_layout, output_area])

# Function to render the chessboard only (without regeneration)
def render_board(board, scale=0.6):
    with chessboard_output:
        clear_output(wait=True)  # Clear only the chessboard output
        board_svg = chess.svg.board(board, size=400 * scale)
        display(SVG(board_svg))

# Function to process user commands (reset, undo, etc.)
def process_command(depth, msg):
    global board, sequence_mode, sequence_moves, current_uci_sequence, current_san_sequence
    msg = msg.strip()

    # Generate the current UCI sequence from the move stack
    current_uci_sequence = [move.uci() for move in board.move_stack]
    current_san_sequence = uci_to_san(current_uci_sequence)  # Convert UCI to SAN

    if sequence_mode:
        sequence_moves.append(msg)
        print(f"Moves in sequence: {sequence_moves}")

        if len(sequence_moves) > 0:
            for move_uci in sequence_moves:
                try:
                    process_moves(move_uci, board)
                    render_board(board)

                    # Fetch and display best move data
                    gm_data = fetch_gm_data(board.fen())
                    best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

                    # Fetch DTZ and WDL data
                    endgame_data = fetch_endgame_tablebase_data(board.fen())
                    wdl = endgame_data.get("WDL", "N/A")
                    dtz = endgame_data.get("DTZ", "N/A")

                    # Fetch best moves from Stockfish and Komodo
                    best_move_stockfish = fetch_stockfish_data(board.fen(), depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
                    best_move_komodo = fetch_komodo_data(board.fen(), depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
                    best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black") or "No endgame move available"

                    # Fetch opening matches
                    opening_matches = check_openings(get_user_moves(board))  # Fetch matching openings

                    # Print out the opening matches
                    if opening_matches:
                        for opening_name, opening_uci in opening_matches:
                            print(f"Matched Opening: {opening_name} (UCI: {opening_uci})")

                    # Display best moves and analysis, along with openings if any
                    display_best_moves_and_analysis(board.fen(), current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

                except ValueError as e:
                    print(f"Invalid move format: {move_uci}. Error: {e}")
            sequence_moves = []
            sequence_mode = False
    elif msg == "quit":
        close_engine()
        sys.exit()
    elif msg == "reset":
        board.reset()
        render_board(board)
        fetch_and_process(board.fen())
        current_uci_sequence = []
        current_san_sequence = []
    elif msg == "undo":
        if len(board.move_stack) > 0:
            board.pop()
        render_board(board)
        fetch_and_process(board.fen())
        current_uci_sequence = [move.uci() for move in board.move_stack]
        current_san_sequence = uci_to_san(current_uci_sequence)
    elif msg == "sequence":
        print("Entering sequence mode. Enter your moves (space-separated UCI moves. For instance, 'e2e4 e7e5 g1f3'):")
        sequence_mode = True
    else:
        try:
            process_moves(msg, board)  # Process the move entered by the user
            render_board(board)

            # Update UCI and SAN sequences after a move
            current_uci_sequence = [move.uci() for move in board.move_stack]
            current_san_sequence = uci_to_san(current_uci_sequence)

            # Fetch and display best move data
            gm_data = fetch_gm_data(board.fen())
            best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

            # Fetch DTZ and WDL data
            endgame_data = fetch_endgame_tablebase_data(board.fen())
            wdl = endgame_data.get("WDL", "N/A")
            dtz = endgame_data.get("DTZ", "N/A")

            # Fetch best moves from Stockfish, Komodo, and Syzygy
            best_move_stockfish = fetch_stockfish_data(board.fen(), depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
            best_move_komodo = fetch_komodo_data(board.fen(), depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
            best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black") or "No endgame move available"

            # Fetch opening matches
            opening_matches = check_openings(get_user_moves(board))  # Fetch matching openings
            if opening_matches:
                for opening_name, opening_uci in opening_matches:
                    print(f"Matched Opening: {opening_name} (UCI: {opening_uci})")

            # Display best moves and analysis, along with openings if any
            display_best_moves_and_analysis(board.fen(), current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

        except ValueError as e:
            print(f"Invalid move format: {msg}. Error: {e}")

# Function to display best move recommendations and analysis in a table format
def fetch_and_process(fen):
    gm_data = fetch_gm_data(fen)
    best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)
    opening_matches = check_openings(get_user_moves(board))
    display_best_moves_and_analysis(fen, current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

def display_best_moves_and_analysis(fen, current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz):
    best_move_stockfish = fetch_stockfish_data(board.fen(), depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
    best_move_komodo = fetch_komodo_data(board.fen(), depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
    best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black")  # Determine turn from board state

    table_rows = [
        ("Current FEN:", fen),
        ("Current UCI Sequence:", current_uci_sequence if current_uci_sequence else "No moves played yet"),
        ("Current SAN Sequence:", " ".join(current_san_sequence) if current_san_sequence else "No moves played yet"),
        ("Matched Opening from Lichess Openings Dataset:", opening_matches if opening_matches else "No matched opening available"),
        ("Best Move from GM Database:", best_move_uci if best_move_uci else "No best move available"),
        ("Top Games with Best Move from GM Database:", top_games if top_games else "No GM games found"),
        ("Best Move from Stockfish:", best_move_stockfish if best_move_stockfish else "No best move available"),
        ("Best Move from Komodo:", best_move_komodo if best_move_komodo else "No best move available"),
        ("Best Endgame Move from Syzygy:", best_move_syzygy if best_move_syzygy else "No endgame move available"),
        ("WDL (Win/Draw/Loss):", wdl),
        ("DTZ (Depth to Zero):", dtz)
    ]

    table_html = """
    <table style='border-collapse: collapse; width: 100%; margin: 0 auto; font-size: 10px;'>
        <tr style='background-color: #f2f2f2;'>
            <th style='border: 0px solid black; padding: 3px 5px; text-align: left; font-weight: bold; font-size: 11px;'>Variable</th>
            <th style='border: 0px solid black; padding: 3px 5px; text-align: left; font-weight: bold; font-size: 11px;'>Value</th>
        </tr>
    """

    # Add table rows with alternating colors for each row
    for index, row in enumerate(table_rows):
        row_color = "#ffffff" if index % 2 == 0 else "#f9f9f9"
        table_html += f"""
        <tr style='background-color: {row_color};'>
            <td style='border: 0px solid black; padding: 2px 5px; font-size: 9px; line-height: 1.2;'>{row[0]}</td>
            <td style='border: 0px solid black; padding: 2px 5px; font-size: 9px; line-height: 1.2;'>{row[1]}</td>
        </tr>
        """

    table_html += "</table>"

    # Display the table below the chessboard (inside the output_area)
    with output_area:
        clear_output(wait=True)
        display(widgets.HTML(value=table_html))

# Update the on_button_click function to pass the best_move_uci
def on_button_click(b):
    msg = input_box.value
    process_command(get_depth(), msg)
    input_box.value = ""  # Clear the input box

# Function to handle reset button click
def on_reset_button_click(b):
    board.reset()
    render_board(board)
    fetch_and_process(board.fen())  # Reset and fetch new best move data
    current_uci_sequence = " ".join([move.uci() for move in board.move_stack])  # Update UCI sequence

# Function to handle undo button click
def on_undo_button_click(b):
    if len(board.move_stack) > 0:
        board.pop()
    render_board(board)
    fetch_and_process(board.fen())  # Fetch and display the best move data after undo
    current_uci_sequence = " ".join([move.uci() for move in board.move_stack])  # Update UCI sequence

def get_depth():
    parser = argparse.ArgumentParser()
    parser.add_argument("--depth", default=3, type=int, help="provide an integer (default: 3)")
    args, unknown = parser.parse_known_args([arg for arg in sys.argv if arg.startswith("--")])
    return max(1, args.depth)

# Function to display the interactive command line and board
def talk():
    submit_button.on_click(on_button_click)  # Bind button click event
    reset_button.on_click(on_reset_button_click)  # Bind reset button event
    undo_button.on_click(on_undo_button_click)  # Bind undo button event
    display(layout_container)  # Display command input, button, and output area
    display(chessboard_output)  # Display the separate chessboard output area
    render_board(board)  # Render the initial board

if __name__ == "__main__":
    talk()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/226k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3507 [00:00<?, ? examples/s]

Output()

Invalid move format: e. Error: expected uci string to be of length 4 or 5: 'e'


In [9]:
import os
import sys
import argparse
import chess
import chess.engine
import chess.svg
from IPython.display import display, SVG, clear_output, Audio, Javascript
import ipywidgets as widgets
import threading
from openings import process_moves, uci_to_san, check_openings, get_user_moves
from gm_database import fetch_gm_data, process_gm_data
from engines import fetch_komodo_data, fetch_stockfish_data
from endgame import fetch_endgame_tablebase_data, best_endgame_move

# ANSI escape code for bold text
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset formatting to normal

# Global variables
current_uci_sequence = []
current_san_sequence = []

# Sequence mode flag and moves
sequence_mode = False
sequence_moves = []
board = chess.Board()  # Initialize the board
is_initial_render = True
chessboard_output = widgets.Output()  # Create a separate output area for rendering the chessboard (this will be updated separately)

# Function to play sound asynchronously to avoid blocking
def play_sound_async(file_path, hide_controls=False):
    try:
        # Play the sound (embed=True avoids showing the player interface)
        display(Audio(file_path, autoplay=True, embed=True))

        if hide_controls:
            # Use JavaScript to hide the audio player interface
            display(Javascript("""
                var audio = document.querySelector('audio');
                if (audio) {
                    audio.style.display = 'none';  // Hide the audio player interface
                    audio.play();  // Ensure the audio is playing
                }
            """))
    except Exception as e:
        print(f"Error playing sound: {e}")

# Create widgets for command input and submit button
input_box = widgets.Text(placeholder='Enter your move or command', description='Command:')
submit_button = widgets.Button(description="Submit")
reset_button = widgets.Button(description="Reset")
undo_button = widgets.Button(description="Undo")
output_area = widgets.Output()

# Layout the buttons horizontally
button_layout = widgets.HBox([submit_button, reset_button, undo_button])

button_style = {'description_width': 'initial'}
submit_button.layout.width = '7%'  # Make the button 5% of the parent container width
reset_button.layout.width = '7%'  # Same for the reset button
undo_button.layout.width = '7%'  # Same for the undo button

submit_button.layout.height = '20px'  # Adjust height to make them shorter
reset_button.layout.height = '20px'  # Same height for reset button
undo_button.layout.height = '20px'  # Same height for undo button

submit_button.style.font = 'Arial 5pt'  # Adjust font size to smaller (8pt)
reset_button.style.font = 'Arial 5pt'  # Adjust font size to smaller (8pt)
undo_button.style.font = 'Arial 5pt'  # Adjust font size to smaller (8pt)

# Optional: Adjust font weight if needed
submit_button.style.font_weight = 'normal'
reset_button.style.font_weight = 'normal'
undo_button.style.font_weight = 'normal'

# Combine input box, buttons, and output area into a vertical layout
layout_container = widgets.VBox([input_box, button_layout, output_area])

# Function to render the chessboard only (without regeneration)
def render_board(board, scale=0.6, sound_file="/content/sample_data/sounds/game_start.mp3", hide_controls=False):
    global is_initial_render
    with chessboard_output:
        clear_output(wait=True)  # Clear only the chessboard output
        board_svg = chess.svg.board(board, size=400 * scale)
        display(SVG(board_svg))
        # Play the sound if it's the initial render or after reset
        if is_initial_render and sound_file:
            play_sound_async(sound_file, hide_controls=True)
            is_initial_render = False

# Function to process user commands (reset, undo, etc.)
def process_command(depth, msg):
    global board, sequence_mode, sequence_moves, current_uci_sequence, current_san_sequence
    msg = msg.strip()

    # Generate the current UCI sequence from the move stack
    current_uci_sequence = [move.uci() for move in board.move_stack]
    current_san_sequence = uci_to_san(current_uci_sequence)  # Convert UCI to SAN

    if sequence_mode:
        sequence_moves.append(msg)
        print(f"Moves in sequence: {sequence_moves}")

        if len(sequence_moves) > 0:
            for move_uci in sequence_moves:
                try:
                    process_moves(move_uci, board)
                    render_board(board)

                    # Fetch and display best move data
                    gm_data = fetch_gm_data(board.fen())
                    best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

                    # Fetch DTZ and WDL data
                    endgame_data = fetch_endgame_tablebase_data(board.fen())
                    wdl = endgame_data.get("WDL", "N/A")
                    dtz = endgame_data.get("DTZ", "N/A")

                    # Fetch best moves from Stockfish and Komodo
                    best_move_stockfish = fetch_stockfish_data(board.fen(), depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
                    best_move_komodo = fetch_komodo_data(board.fen(), depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
                    best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black") or "No endgame move available"

                    # Fetch opening matches
                    opening_matches = check_openings(get_user_moves(board))  # Fetch matching openings

                    # Print out the opening matches
                    if opening_matches:
                        for opening_name, opening_uci in opening_matches:
                            print(f"Matched Opening: {opening_name} (UCI: {opening_uci})")

                    # Display best moves and analysis, along with openings if any
                    display_best_moves_and_analysis(board.fen(), current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

                except ValueError as e:
                    print(f"Invalid move format: {move_uci}. Error: {e}")
            sequence_moves = []
            sequence_mode = False
    elif msg == "quit":
        close_engine()
        sys.exit()
    elif msg == "reset":
        board.reset()
        render_board(board)
        fetch_and_process(board.fen())
        current_uci_sequence = []
        current_san_sequence = []
    elif msg == "undo":
        if len(board.move_stack) > 0:
            board.pop()
        render_board(board)
        fetch_and_process(board.fen())
        current_uci_sequence = [move.uci() for move in board.move_stack]
        current_san_sequence = uci_to_san(current_uci_sequence)
    elif msg == "sequence":
        print("Entering sequence mode. Enter your moves (space-separated UCI moves. For instance, 'e2e4 e7e5 g1f3'):")
        sequence_mode = True
    else:
        try:
            process_moves(msg, board)  # Process the move entered by the user
            render_board(board)

            # Update UCI and SAN sequences after a move
            current_uci_sequence = [move.uci() for move in board.move_stack]
            current_san_sequence = uci_to_san(current_uci_sequence)

            # Fetch and display best move data
            gm_data = fetch_gm_data(board.fen())
            best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

            # Fetch DTZ and WDL data
            endgame_data = fetch_endgame_tablebase_data(board.fen())
            wdl = endgame_data.get("WDL", "N/A")
            dtz = endgame_data.get("DTZ", "N/A")

            # Fetch best moves from Stockfish, Komodo, and Syzygy
            best_move_stockfish = fetch_stockfish_data(board.fen(), depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
            best_move_komodo = fetch_komodo_data(board.fen(), depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
            best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black") or "No endgame move available"

            # Fetch opening matches
            opening_matches = check_openings(get_user_moves(board))  # Fetch matching openings
            if opening_matches:
                for opening_name, opening_uci in opening_matches:
                    print(f"Matched Opening: {opening_name} (UCI: {opening_uci})")

            # Display best moves and analysis, along with openings if any
            display_best_moves_and_analysis(board.fen(), current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

        except ValueError as e:
            print(f"Invalid move format: {msg}. Error: {e}")

# Function to display best move recommendations and analysis in a table format
def fetch_and_process(fen):
    # Fetch GM data and process
    gm_data = fetch_gm_data(fen)
    best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

    # Fetch best moves from Stockfish, Komodo, and Syzygy
    best_move_stockfish = fetch_stockfish_data(fen, depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
    best_move_komodo = fetch_komodo_data(fen, depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
    best_move_syzygy = best_endgame_move(fen, turn="white" if board.turn else "black") or "No endgame move available"

    # Fetch opening matches
    opening_matches = check_openings(get_user_moves(board))

    # Fetch endgame data (WDL, DTZ)
    endgame_data = fetch_endgame_tablebase_data(fen)
    wdl = endgame_data.get("WDL", "N/A")
    dtz = endgame_data.get("DTZ", "N/A")

    # Display the analysis
    display_best_moves_and_analysis(fen, current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

def display_best_moves_and_analysis(fen, current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz):
    best_move_stockfish = fetch_stockfish_data(board.fen(), depth=15, stockfish_path="/usr/games/stockfish") or "No best move available"
    best_move_komodo = fetch_komodo_data(board.fen(), depth=15, komodo_path="/content/komodo3sse42") or "No best move available"
    best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black")  # Determine turn from board state

    table_rows = [
        ("Current FEN:", fen),
        ("Current UCI Sequence:", current_uci_sequence if current_uci_sequence else "No moves played yet"),
        ("Current SAN Sequence:", " ".join(current_san_sequence) if current_san_sequence else "No moves played yet"),
        ("Matched Opening from Lichess Openings Dataset:", opening_matches if opening_matches else "No matched opening available"),
        ("Best Move from GM Database:", best_move_uci if best_move_uci else "No best move available"),
        ("Top Games with Best Move from GM Database:", top_games if top_games else "No GM games found"),
        ("Best Move from Stockfish:", best_move_stockfish if best_move_stockfish else "No best move available"),
        ("Best Move from Komodo:", best_move_komodo if best_move_komodo else "No best move available"),
        ("Best Endgame Move from Syzygy:", best_move_syzygy if best_move_syzygy else "No endgame move available"),
        ("WDL (Win/Draw/Loss):", wdl),
        ("DTZ (Depth to Zero):", dtz)
    ]

    table_html = """
    <table style='border-collapse: collapse; width: 100%; margin: 0 auto; font-size: 10px;'>
        <tr style='background-color: #f2f2f2;'>
            <th style='border: 0px solid black; padding: 3px 5px; text-align: left; font-weight: bold; font-size: 11px;'>Variable</th>
            <th style='border: 0px solid black; padding: 3px 5px; text-align: left; font-weight: bold; font-size: 11px;'>Value</th>
        </tr>
    """

    # Add table rows with alternating colors for each row
    for index, row in enumerate(table_rows):
        row_color = "#ffffff" if index % 2 == 0 else "#f9f9f9"
        table_html += f"""
        <tr style='background-color: {row_color};'>
            <td style='border: 0px solid black; padding: 2px 5px; font-size: 9px; line-height: 1.2;'>{row[0]}</td>
            <td style='border: 0px solid black; padding: 2px 5px; font-size: 9px; line-height: 1.2;'>{row[1]}</td>
        </tr>
        """

    table_html += "</table>"

    # Display the table below the chessboard (inside the output_area)
    with output_area:
        clear_output(wait=True)
        display(widgets.HTML(value=table_html))

# Update the on_button_click function to pass the best_move_uci
def on_button_click(b):
    msg = input_box.value
    process_command(get_depth(), msg)
    input_box.value = ""  # Clear the input box

# Function to handle reset button click
def on_reset_button_click(b):
    global is_initial_render
    is_initial_render = True  # Reset the flag before rendering the board
    board.reset()  # Reset the board to the initial state
    render_board(board, sound_file="/content/sample_data/sounds/game_start.mp3")  # Render the board with hidden controls

    # Forcefully hide the audio player interface after reset, just in case
    display(Javascript("""
        var audio = document.querySelector('audio');
        if (audio) {
            audio.style.display = 'none';  // Hide the audio player interface
        }
    """))

    fetch_and_process(board.fen())  # Reset and fetch new best move data
    current_uci_sequence = " ".join([move.uci() for move in board.move_stack])  # Update UCI sequence

# Function to handle undo button click
def on_undo_button_click(b):
    if len(board.move_stack) > 0:
        board.pop()
    render_board(board)
    fetch_and_process(board.fen())  # Fetch and display the best move data after undo
    current_uci_sequence = " ".join([move.uci() for move in board.move_stack])  # Update UCI sequence

def get_depth():
    parser = argparse.ArgumentParser()
    parser.add_argument("--depth", default=3, type=int, help="provide an integer (default: 3)")
    args, unknown = parser.parse_known_args([arg for arg in sys.argv if arg.startswith("--")])
    return max(1, args.depth)

# Function to display the interactive command line and board
def talk():
    submit_button.on_click(on_button_click)  # Bind button click event
    reset_button.on_click(on_reset_button_click)  # Bind reset button event
    undo_button.on_click(on_undo_button_click)  # Bind undo button event
    display(layout_container)  # Display command input, button, and output area
    display(chessboard_output)  # Display the separate chessboard output area
    render_board(board)  # Render the initial board

if __name__ == "__main__":
    talk()

Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>